<a href="https://colab.research.google.com/github/OANInternational/handicape/blob/master/asphan_estructurar_bbdd_y_subir_a_firebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto para estructurar base de datos y subir a FIRBASE

# INTSTALS

In [0]:
#!pip install firebase-admin
#!pip install geocoder
#!pip install fuzzywuzzy[speedup]

# DIRECTORIO

utilizar google como directorio de trabajo


In [4]:
%cd

/root


In [5]:
%cd
%cd ..
%cd content

/root
/
/content


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd drive/'Shared drives'/'C. Políticas Sociales'/'PROYECTOS'/'Handicapées'

/content/drive/Shared drives/C. Políticas Sociales/PROYECTOS/Handicapées


In [0]:
import yaml
spreadsheet_ids = yaml.load( open("ASPHAN-APP/sheets_parameters.yaml", "r" ))
#recensement de personnes handicapées
ID=str(spreadsheet_ids['ID'])


# CONNECT TO FIREBASE

In [0]:
filename='ASPHAN-APP/firebase_parameters.json'

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import storage

# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

# ABRIR LAS BBDD

## OPEN GOOGLE SHEETS

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
sheet_service = build('sheets', 'v4')

## IMPORTS

In [0]:
import numpy as np
import re

import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## OPEN FROM GOOGLE SHEET

In [0]:
RANGE_NAME = 'handicapees'
gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()
df_hand = pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0])

In [49]:
df_hand.shape

(461, 50)

### PROCESS DATA

**Change age to integers**

In [0]:
ages = df_hand['Age'].to_list()
for i,age in enumerate(ages):
  if age != "":
    ages[i]=int(age)
  else:
    ages[i] = None

In [0]:
df_hand['Age']=ages

In [0]:
df_hand['Nombre du form']=list(map(int,df_hand['Nombre du form'].to_list()))

**Convert profession into similar**

In [0]:
words = ["menagère","Couturiere","Revendeuse","Forgeron","Meunuisier","Écolier"]
professions = df_hand['Profession'].to_list()
for i,prof in enumerate(professions):
  for word in words:
    if fuzz.ratio(word, prof) >60:
      professions[i]=word

In [0]:
df_hand['Profession']=professions

In [0]:
professions_part = df_hand['Activites du partenaire'].to_list()
for i,prof in enumerate(professions_part):
  for word in words:
    if fuzz.ratio(word, prof) >60:
      professions_part[i]=word

In [0]:
df_hand['Activites du partenaire']=professions_part

### UPLOAD TO FIREBASE

### Add images to firestore

In [0]:
storage = firebase_admin.storage

In [0]:
bucket = storage.bucket(XXX)

**I will upload only the important**

In [0]:
handicapees = df_hand.to_dict('index')
hand_dict = [{k: v} for k, v in handicapees.items()] 
for hand in hand_dict:
  id_hand=list(hand.keys())[0]
  handicape=hand[id_hand]
  n_photo1=handicape['Photo 1']
  n_photo2=handicape['Photo 2']
  if((n_photo1 != "") and (n_photo1 is not None)):
    photo1_blob = bucket.blob('phots_hand/'+n_photo1)
    photo1_blob.upload_from_filename('Recensement/photos_hand/'+n_photo1)
    photo1_blob.make_public()
    photo1_url=photo1_blob.public_url

    photo2_blob = bucket.blob('phots_hand/'+n_photo2)
    photo2_blob.upload_from_filename('Recensement/photos_hand/'+n_photo2)
    photo2_blob.make_public()
    photo2_url=photo2_blob.public_url
  else:
    photo1_url=None
    photo2_url=None
  
  new_data={'code':handicape['Nombre du form'],
            'arrondisement': handicape['Arrondisement'],
            'village':handicape['Village'],
            'nom':handicape['Nom'],
            'prenoms':handicape['Prénoms'],
            'age':handicape['Age'],
            'sex':handicape['Sex'],
            'type':handicape['Type de Handicappées'],
            'besoin_sociale':handicape['Besoin Sociale'],
            'besoin_technique':handicape['Besoin Technique'],
            'profession':handicape['Profession'],
            'images':{
                'image1':{
                    'name':n_photo1,
                    'url':photo1_url
                          },
                'image2':{
                    'name':n_photo2,
                    'url':photo2_url
                         }
                     }
             }
  db.collection(u'HandicapeesDB').add(new_data)